In [133]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from sqlalchemy import create_engine
from splinter import Browser
import time
from config import username, password

In [2]:
food_file = "resources/emissions_by_food.csv"
food_df = pd.read_csv(food_file)
food_df


,Food product,Land use change,Animal Feed,Farm,Processing,Transport,Packging,Retail,Unnamed: 8
0,Wheat & Rye (Bread),0.1,0.0,0.8,0.2,0.1,0.1,0.1,NaN
1,Maize (Meal),0.3,0.0,0.5,0.1,0.1,0.1,0.0,NaN
2,Barley (Beer),0.0,0.0,0.2,0.1,0.0,0.5,0.3,NaN
3,Oatmeal,0.0,0.0,1.4,0.0,0.1,0.1,0.0,NaN
4,Rice,0.0,0.0,3.6,0.1,0.1,0.1,0.1,NaN
5,Potatoes,0.0,0.0,0.2,0.0,0.1,0.0,0.0,NaN
6,Cassava,0.6,0.0,0.2,0.0,0.1,0.0,0.0,NaN
7,Cane Sugar,1.2,0.0,0.5,0.0,0.8,0.1,0.0,NaN
8,Beet Sugar,0.0,0.0,0.5,0.2,0.6,0.1,0.0,NaN
9,Other Pulses,0.0,0.0,1.1,0.0,0.1,0.4,0.0,NaN


In [3]:
del food_df["Unnamed: 8"]
food_df.head()

,Food product,Land use change,Animal Feed,Farm,Processing,Transport,Packging,Retail
0,Wheat & Rye (Bread),0.1,0.0,0.8,0.2,0.1,0.1,0.1
1,Maize (Meal),0.3,0.0,0.5,0.1,0.1,0.1,0.0
2,Barley (Beer),0.0,0.0,0.2,0.1,0.0,0.5,0.3
3,Oatmeal,0.0,0.0,1.4,0.0,0.1,0.1,0.0
4,Rice,0.0,0.0,3.6,0.1,0.1,0.1,0.1


In [4]:
food_df["Food product"] = food_df["Food product"].replace({"Milk":"Dairy", "Cheese":"Dairy", "Wheat & Rye (Bread)":"Wheat and Wheat Products", "Groundnuts":"Nuts"})
food_df


,Food product,Land use change,Animal Feed,Farm,Processing,Transport,Packging,Retail
0,Wheat and Wheat Products,0.1,0.0,0.8,0.2,0.1,0.1,0.1
1,Maize (Meal),0.3,0.0,0.5,0.1,0.1,0.1,0.0
2,Barley (Beer),0.0,0.0,0.2,0.1,0.0,0.5,0.3
3,Oatmeal,0.0,0.0,1.4,0.0,0.1,0.1,0.0
4,Rice,0.0,0.0,3.6,0.1,0.1,0.1,0.1
5,Potatoes,0.0,0.0,0.2,0.0,0.1,0.0,0.0
6,Cassava,0.6,0.0,0.2,0.0,0.1,0.0,0.0
7,Cane Sugar,1.2,0.0,0.5,0.0,0.8,0.1,0.0
8,Beet Sugar,0.0,0.0,0.5,0.2,0.6,0.1,0.0
9,Other Pulses,0.0,0.0,1.1,0.0,0.1,0.4,0.0


In [5]:
food_df = food_df.groupby("Food product").sum()
food_df


,Land use change,Animal Feed,Farm,Processing,Transport,Packging,Retail
Food product,,,,,,,
Apples,0.0,0.0,0.2,0.0,0.1,0.0,0.0
Bananas,0.0,0.0,0.3,0.1,0.3,0.1,0.0
Barley (Beer),0.0,0.0,0.2,0.1,0.0,0.5,0.3
Beef (beef herd),16.3,1.9,39.4,1.3,0.3,0.2,0.2
Beef (dairy herd),0.9,2.5,15.7,1.1,0.4,0.3,0.2
Beet Sugar,0.0,0.0,0.5,0.2,0.6,0.1,0.0
Berries & Grapes,0.0,0.0,0.7,0.0,0.2,0.2,0.0
Brassicas,0.0,0.0,0.3,0.0,0.1,0.0,0.0
Cane Sugar,1.2,0.0,0.5,0.0,0.8,0.1,0.0


In [6]:
food_df = food_df.reset_index()
food_df


,Food product,Land use change,Animal Feed,Farm,Processing,Transport,Packging,Retail
0,Apples,0.0,0.0,0.2,0.0,0.1,0.0,0.0
1,Bananas,0.0,0.0,0.3,0.1,0.3,0.1,0.0
2,Barley (Beer),0.0,0.0,0.2,0.1,0.0,0.5,0.3
3,Beef (beef herd),16.3,1.9,39.4,1.3,0.3,0.2,0.2
4,Beef (dairy herd),0.9,2.5,15.7,1.1,0.4,0.3,0.2
5,Beet Sugar,0.0,0.0,0.5,0.2,0.6,0.1,0.0
6,Berries & Grapes,0.0,0.0,0.7,0.0,0.2,0.2,0.0
7,Brassicas,0.0,0.0,0.3,0.0,0.1,0.0,0.0
8,Cane Sugar,1.2,0.0,0.5,0.0,0.8,0.1,0.0
9,Cassava,0.6,0.0,0.2,0.0,0.1,0.0,0.0


In [7]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)
# executable_path = {'executable_path': 'chromedriver.exe'}
# browser = Browser('chrome', **executable_path, headless=False)


In [8]:
url = 'https://www.nu3.de/blogs/nutrition/food-carbon-footprint-index-2018'
# connects browser to desired webpage
browser.visit(url)
time.sleep(1)
html = browser.html

In [9]:
# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(html, 'html.parser')


In [10]:
tables = pd.read_html(url)
tables


[    Unnamed: 0_level_0 Unnamed: 1_level_0    Animal Products  \
     Unnamed: 0_level_1 Unnamed: 1_level_1 Unnamed: 2_level_1   
                      #            Country Unnamed: 2_level_2   
 0                    1          Argentina              10.51   
 1                    2          Australia              24.14   
 2                    3            Albania              10.88   
 3                    4            Iceland              21.69   
 4                    5        New Zealand              22.29   
 ..                 ...                ...                ...   
 125                126       Sierra Leone               1.22   
 126                127          Sri Lanka               0.05   
 127                128          Indonesia               2.98   
 128                129            Liberia               4.01   
 129                130         Bangladesh               0.00   
 
                                                               \
     Unnamed: 3_level_1

In [11]:
countries_df = tables[0]
countries_df


Unnamed: 0_level_0 Unnamed: 1_level_0    Animal Products  \
    Unnamed: 0_level_1 Unnamed: 1_level_1 Unnamed: 2_level_1   
                     #            Country Unnamed: 2_level_2   
0                    1          Argentina              10.51   
1                    2          Australia              24.14   
2                    3            Albania              10.88   
3                    4            Iceland              21.69   
4                    5        New Zealand              22.29   
..                 ...                ...                ...   
125                126       Sierra Leone               1.22   
126                127          Sri Lanka               0.05   
127                128          Indonesia               2.98   
128                129            Liberia               4.01   
129                130         Bangladesh               0.00   

                                                              \
    Unnamed: 3_level_1 Unnamed: 4_level_1 Unnamed: 5_level_1   
    Unnamed: 3_level_2 Unnamed: 4_level_2 Unnamed: 5_level_2   
0                37.20              38.66              41.53   
1                85.44              46.12              49.54   
2                38.51              13.23              14.21   
3                76.77              26.87              28.86   
4                78.90              34.98              37.58   
..                 ...                ...                ...   
125               4.32               4.97               5.34   
126               0.18               4.96               5.33   
127              10.55               7.50               8.06   
128              14.19               8.91               9.57   
129               0.00               1.40               1.50   

                                                              \
    Unnamed: 6_level_1 Unnamed: 7_level_1 Unnamed: 8_level_1   
    Unnamed: 6_level_2 Unnamed: 7_level_2 Unnamed: 8_level_2   
0                55.48            1712.00               1.56   
1                33.86            1044.85               9.87   
2                22.50             694.30              15.32   
3                13.36             412.26              21.12   
4                22.49             693.99              18.91   
..                 ...                ...                ...   
125               1.99              61.41               0.65   
126               1.38              42.58               0.09   
127               2.59              79.92               0.46   
128               0.78              24.07               0.48   
129               1.28              39.50               1.33   

                        ... Non-Animal Products                      \
    Unnamed: 9_level_1  ... Unnamed: 17_level_1 Unnamed: 18_level_1   
    Unnamed: 9_level_2  ... Unnamed: 17_level_2 Unnamed: 18_level_2   
0                54.63  ...              103.11               19.66   
1               345.65  ...               70.46               13.44   
2               536.50  ...              138.64               26.44   
3               739.62  ...               72.92               13.91   
4               662.23  ...               76.91               14.67   
..                 ...  ...                 ...                 ...   
125              22.76  ...                8.72                1.66   
126               3.15  ...               37.01                7.06   
127              16.11  ...               25.45                4.85   
128              16.81  ...               10.95                2.09   
129              46.58  ...               17.47                3.33   

                                                                 \
    Unnamed: 19_level_1 Unnamed: 20_level_1 Unnamed: 21_level_1   
    Unnamed: 19_level_2 Unnamed: 20_level_2 Unnamed: 21_level_2   
0                  8.77               11.22                0.00   
1                 11.03               14.12       

In [12]:
del countries_df["Unnamed: 0_level_0"]
countries_df


Unnamed: 1_level_0    Animal Products                     \
    Unnamed: 1_level_1 Unnamed: 2_level_1 Unnamed: 3_level_1   
               Country Unnamed: 2_level_2 Unnamed: 3_level_2   
0            Argentina              10.51              37.20   
1            Australia              24.14              85.44   
2              Albania              10.88              38.51   
3              Iceland              21.69              76.77   
4          New Zealand              22.29              78.90   
..                 ...                ...                ...   
125       Sierra Leone               1.22               4.32   
126          Sri Lanka               0.05               0.18   
127          Indonesia               2.98              10.55   
128            Liberia               4.01              14.19   
129         Bangladesh               0.00               0.00   

                                                              \
    Unnamed: 4_level_1 Unnamed: 5_level_1 Unnamed: 6_level_1   
    Unnamed: 4_level_2 Unnamed: 5_level_2 Unnamed: 6_level_2   
0                38.66              41.53              55.48   
1                46.12              49.54              33.86   
2                13.23              14.21              22.50   
3                26.87              28.86              13.36   
4                34.98              37.58              22.49   
..                 ...                ...                ...   
125               4.97               5.34               1.99   
126               4.96               5.33               1.38   
127               7.50               8.06               2.59   
128               8.91               9.57               0.78   
129               1.40               1.50               1.28   

                                                              \
    Unnamed: 7_level_1 Unnamed: 8_level_1 Unnamed: 9_level_1   
    Unnamed: 7_level_2 Unnamed: 8_level_2 Unnamed: 9_level_2   
0              1712.00               1.56              54.63   
1              1044.85               9.87             345.65   
2               694.30              15.32             536.50   
3               412.26              21.12             739.62   
4               693.99              18.91             662.23   
..                 ...                ...                ...   
125              61.41               0.65              22.76   
126              42.58               0.09               3.15   
127              79.92               0.46              16.11   
128              24.07               0.48              16.81   
129              39.50               1.33              46.58   

                         ... Non-Animal Products                      \
    Unnamed: 10_level_1  ... Unnamed: 17_level_1 Unnamed: 18_level_1   
    Unnamed: 10_level_2  ... Unnamed: 17_level_2 Unnamed: 18_level_2   
0                  4.36  ...              103.11               19.66   
1                 17.69  ...               70.46               13.44   
2                  3.85  ...              138.64               26.44   
3                 74.41  ...               72.92               13.91   
4                 20.36  ...               76.91               14.67   
..                  ...  ...                 ...                 ...   
125               32.03  ...                8.72                1.66   
126               24.92  ...               37.01                7.06   
127               25.65  ...               25.45                4.85   
128                4.13  ...               10.95                2.09   
129               18.07  ...               17.47                3.33   

                                                                 \
    Unnamed: 19_level_1 Unnamed: 20_level_1 Unnamed: 21_level_1   
    Unnamed: 19_level_2 Unnamed: 20_level_2 Unnamed: 21_level_2   
0                  8.77               11.22                0.00   
1                 11.03             

In [13]:
# df = df.rename(columns={"Unnamed: 1_level_0":"", "Unnamed: 1_level_1":"", "Unnamed: 2_level_1":"Pig Meat", "Unnamed: 3_level_1":"Pig Meat", "Unnamed: 4_level_1":"Poultry Meat", "Unnamed: 5_level_1":"Poultry Meat", "Unnamed: 6_level_1":"Beef (beef herd)", "Unnamed: 7_level_1": "Beef (beef herd)", "Unnamed: 8_level_1":"Lamb & Mutton", "Unnamed: 9_level_1":"Lamb & Mutton", "Unnamed: 10_level_1":"Fish (farmed)", "Unnamed: 11_level_1":"Fish (farmed)",
#                        "Unnamed: 12_level_1":"Eggs", "Unnamed: 13_level_1":"Eggs", "Unnamed: 14_level_1":"Dairy", "Unnamed: 15_level_1": "Dairy", "Unnamed: 16_level_1": "",
#                        "Unnamed: 17_level_1": "Wheat and Wheat Products", "Unnamed: 18_level_1": "Wheat and Wheat Products", "Unnamed: 19_level_1": "Rice", "Unnamed: 20_level_1": "Rice", "Unnamed: 21_level_1": "Soybeans", "Unnamed: 22_level_1": "Soybeans", "Unnamed: 23_level_1": "Nuts", "Unnamed: 24_level_1": "Nuts", "Unnamed: 25_level_1": "", "Unnamed: 26_level_2": "",
#                        "Unnamed: 2_level_2":"Supplied for Consumption", "Unnamed: 4_level_2":"Supplied for Consumption", "Unnamed: 6_level_2":"Supplied for Consumption", "Unnamed: 8_level_2":"Supplied for Consumption", "Unnamed: 10_level_2":"Supplied for Consumption",
#                        "Unnamed: 12_level_2":"Supplied for Consumption", "Unnamed: 14_level_2":"Supplied for Consumption", "Unnamed: 16_level_2":"Total Kg CO2/person/year for Animal Products",
#                        "Unnamed: 17_level_2":"Supplied for Consumption", "Unnamed: 19_level_2":"Supplied for Consumption", "Unnamed: 21_level_2":"Supplied for Consumption", "Unnamed: 23_level_2":"Supplied for Consumption", "Unnamed: 25_level_2":"Total Kg CO2/person/year for Non-Animal Products"})
# df

countries_df = countries_df.rename(columns={"Unnamed: 1_level_0":"", "Unnamed: 1_level_1":"", "Unnamed: 2_level_1":"Pig Meat", "Unnamed: 3_level_1":"Pig Meat", "Unnamed: 4_level_1":"Poultry Meat", "Unnamed: 5_level_1":"Poultry Meat", "Unnamed: 6_level_1":"Beef (beef herd)", "Unnamed: 7_level_1": "Beef (beef herd)", "Unnamed: 8_level_1":"Lamb & Mutton", "Unnamed: 9_level_1":"Lamb & Mutton", "Unnamed: 10_level_1":"Fish (farmed)", "Unnamed: 11_level_1":"Fish (farmed)",
                                           "Unnamed: 12_level_1":"Eggs", "Unnamed: 13_level_1":"Eggs", "Unnamed: 14_level_1":"Dairy", "Unnamed: 15_level_1": "Dairy", "Unnamed: 16_level_1": "",
                                           "Unnamed: 17_level_1": "Wheat and Wheat Products", "Unnamed: 18_level_1": "Wheat and Wheat Products", "Unnamed: 19_level_1": "Rice", "Unnamed: 20_level_1": "Rice", "Unnamed: 21_level_1": "Soybeans", "Unnamed: 22_level_1": "Soybeans", "Unnamed: 23_level_1": "Nuts", "Unnamed: 24_level_1": "Nuts", "Unnamed: 25_level_1": "", "Unnamed: 26_level_2": "",
                                           "Unnamed: 2_level_2":"Supplied for Consumption", "Unnamed: 3_level_2":"Actually Consumed", "Unnamed: 4_level_2":"Supplied for Consumption", "Unnamed: 5_level_2":"Actually Consumed", "Unnamed: 6_level_2":"Supplied for Consumption", "Unnamed: 7_level_2":"Actually Consumed", "Unnamed: 8_level_2":"Supplied for Consumption", "Unnamed: 9_level_2":"Actually Consumed", "Unnamed: 10_level_2":"Supplied for Consumption", "Unnamed: 11_level_2":"Actually Consumed",
                                           "Unnamed: 12_level_2":"Supplied for Consumption", "Unnamed: 13_level_2":"Actually Consumed", "Unnamed: 14_level_2":"Supplied for Consumption", "Unnamed: 15_level_2":"Actually Consumed", "Unnamed: 16_level_2":"Total Kg CO2/person/year for Animal Products",
                                           "Unnamed: 17_level_2":"Supplied for Consumption", "Unnamed: 18_level_2":"Actually Consumed", "Unnamed: 19_level_2":"Supplied for Consumption", "Unnamed: 20_level_2":"Actually Consumed", "Unnamed: 21_level_2":"Supplied for Consumption", "Unnamed: 22_level_2":"Actually Consumed", "Unnamed: 23_level_2":"Supplied for Consumption", "Unnamed: 24_level_2":"Actually Consumed", "Unnamed: 25_level_2":"Total Kg CO2/person/year for Non-Animal Products"})
countries_df


Animal Products                    \
                                  Pig Meat                     
          Country Supplied for Consumption Actually Consumed   
0       Argentina                    10.51             37.20   
1       Australia                    24.14             85.44   
2         Albania                    10.88             38.51   
3         Iceland                    21.69             76.77   
4     New Zealand                    22.29             78.90   
..            ...                      ...               ...   
125  Sierra Leone                     1.22              4.32   
126     Sri Lanka                     0.05              0.18   
127     Indonesia                     2.98             10.55   
128       Liberia                     4.01             14.19   
129    Bangladesh                     0.00              0.00   

                                                                         \
                Poultry Meat                           Beef (beef herd)   
    Supplied for Consumption Actually Consumed Supplied for Consumption   
0                      38.66             41.53                    55.48   
1                      46.12             49.54                    33.86   
2                      13.23             14.21                    22.50   
3                      26.87             28.86                    13.36   
4                      34.98             37.58                    22.49   
..                       ...               ...                      ...   
125                     4.97              5.34                     1.99   
126                     4.96              5.33                     1.38   
127                     7.50              8.06                     2.59   
128                     8.91              9.57                     0.78   
129                     1.40              1.50                     1.28   

                                                                  \
                                 Lamb & Mutton                     
    Actually Consumed Supplied for Consumption Actually Consumed   
0             1712.00                     1.56             54.63   
1             1044.85                     9.87            345.65   
2              694.30                    15.32            536.50   
3              412.26                    21.12            739.62   
4              693.99                    18.91            662.23   
..                ...                      ...               ...   
125             61.41                     0.65             22.76   
126             42.58                     0.09              3.15   
127             79.92                     0.46             16.11   
128             24.07                     0.48             16.81   
129             39.50                     1.33             46.58   

                              ...      Non-Animal Products                    \
               Fish (farmed)  ... Wheat and Wheat Products                     
    Supplied for Consumption  ... Supplied for Consumption Actually Consumed   
0                       4.36  ...                   103.11             19.66   
1                      17.69  ...                    70.46             13.44   
2                       3.85  ...                   138.64             26.44   
3                      74.41  ...                    72.92             13.91   
4                      20.36  ...                    76.91             14.67   
..                       ...  ...                      ...               ...   
125                    32.03  ...                     8.72              1.66   
126                    24.92  ...                    37.01              7.06   
127                    25.65  ...                    25.45              4.85   
128                     4.13  ...                    10.95              2.09   
129                    18.07  ...                    17.47              3.33   

     

In [14]:
countries_df = countries_df.transpose()
countries_df.head()


0          1    \
                             Country                   Argentina  Australia   
Animal Products Pig Meat     Supplied for Consumption      10.51      24.14   
                             Actually Consumed              37.2      85.44   
                Poultry Meat Supplied for Consumption      38.66      46.12   
                             Actually Consumed             41.53      49.54   

                                                           2        3    \
                             Country                   Albania  Iceland   
Animal Products Pig Meat     Supplied for Consumption    10.88    21.69   
                             Actually Consumed           38.51    76.77   
                Poultry Meat Supplied for Consumption    13.23    26.87   
                             Actually Consumed           14.21    28.86   

                                                               4      5    \
                             Country                   New Zealand    USA   
Animal Products Pig Meat     Supplied for Consumption        22.29  27.64   
                             Actually Consumed                78.9  97.83   
                Poultry Meat Supplied for Consumption        34.98  50.01   
                             Actually Consumed               37.58  53.72   

                                                           6           7    \
                             Country                   Uruguay  Luxembourg   
Animal Products Pig Meat     Supplied for Consumption    16.84       43.58   
                             Actually Consumed           59.61      154.25   
                Poultry Meat Supplied for Consumption    27.45       21.37   
                             Actually Consumed           29.49       22.96   

                                                          8           9    \
                             Country                   Brazil  Kazakhstan   
Animal Products Pig Meat     Supplied for Consumption    12.6       10.36   
                             Actually Consumed           44.6       36.67   
                Poultry Meat Supplied for Consumption      45       18.38   
                             Actually Consumed          48.34       19.74   

                                                       ...     120    121  \
                             Country                   ...  Gambia  India   
Animal Products Pig Meat     Supplied for Consumption  ...    0.18   0.28   
                             Actually Consumed         ...    0.64   0.99   
                Poultry Meat Supplied for Consumption  ...    2.66   1.88   
                             Actually Consumed         ...    2.86   2.02   

                                                            122         123  \
                             Country                   Thailand  Mozambique   
Animal Products Pig Meat     Supplied for Consumption        13        5.13   
                             Actually Consumed            46.01       18.16   
                Poultry Meat Supplied for Consumption     13.69        1.41   
                             Actually Consumed            14.71        1.51   

                                                            124           125  \
                             Country                   Cambodia  Sierra Leone   
Animal Products Pig Meat     Supplied for Consumption      6.52          1.22   
                             Actually Consumed            23.08          4.32   
                Poultry Meat Supplied for Consumption      1.84          4.97   
                             Actually Consumed             1.98          5.34   

                                                             126        127  \
                             Country                   Sri Lanka  Indonesia   
Animal Products Pig Meat     Supplied for Consumption       0.05       2.98   
                             Actually Consumed              0.18      10.55   

In [15]:
countries_df = countries_df.reset_index()
countries_df.head()


,level_0,level_1,level_2,0,1,2,3,4,5,6,...,120,121,122,123,124,125,126,127,128,129
0,,,Country,Argentina,Australia,Albania,Iceland,New Zealand,USA,Uruguay,...,Gambia,India,Thailand,Mozambique,Cambodia,Sierra Leone,Sri Lanka,Indonesia,Liberia,Bangladesh
1,Animal Products,Pig Meat,Supplied for Consumption,10.51,24.14,10.88,21.69,22.29,27.64,16.84,...,0.18,0.28,13,5.13,6.52,1.22,0.05,2.98,4.01,0
2,Animal Products,Pig Meat,Actually Consumed,37.2,85.44,38.51,76.77,78.9,97.83,59.61,...,0.64,0.99,46.01,18.16,23.08,4.32,0.18,10.55,14.19,0
3,Animal Products,Poultry Meat,Supplied for Consumption,38.66,46.12,13.23,26.87,34.98,50.01,27.45,...,2.66,1.88,13.69,1.41,1.84,4.97,4.96,7.5,8.91,1.4
4,Animal Products,Poultry Meat,Actually Consumed,41.53,49.54,14.21,28.86,37.58,53.72,29.49,...,2.86,2.02,14.71,1.51,1.98,5.34,5.33,8.06,9.57,1.5


In [16]:
del countries_df["level_0"]
countries_df.head()


,level_1,level_2,0,1,2,3,4,5,6,7,...,120,121,122,123,124,125,126,127,128,129
0,,Country,Argentina,Australia,Albania,Iceland,New Zealand,USA,Uruguay,Luxembourg,...,Gambia,India,Thailand,Mozambique,Cambodia,Sierra Leone,Sri Lanka,Indonesia,Liberia,Bangladesh
1,Pig Meat,Supplied for Consumption,10.51,24.14,10.88,21.69,22.29,27.64,16.84,43.58,...,0.18,0.28,13,5.13,6.52,1.22,0.05,2.98,4.01,0
2,Pig Meat,Actually Consumed,37.2,85.44,38.51,76.77,78.9,97.83,59.61,154.25,...,0.64,0.99,46.01,18.16,23.08,4.32,0.18,10.55,14.19,0
3,Poultry Meat,Supplied for Consumption,38.66,46.12,13.23,26.87,34.98,50.01,27.45,21.37,...,2.66,1.88,13.69,1.41,1.84,4.97,4.96,7.5,8.91,1.4
4,Poultry Meat,Actually Consumed,41.53,49.54,14.21,28.86,37.58,53.72,29.49,22.96,...,2.86,2.02,14.71,1.51,1.98,5.34,5.33,8.06,9.57,1.5


In [17]:
countries_df.iloc[0,0] = "Food product"
countries_df.head()


,level_1,level_2,0,1,2,3,4,5,6,7,...,120,121,122,123,124,125,126,127,128,129
0,Food product,Country,Argentina,Australia,Albania,Iceland,New Zealand,USA,Uruguay,Luxembourg,...,Gambia,India,Thailand,Mozambique,Cambodia,Sierra Leone,Sri Lanka,Indonesia,Liberia,Bangladesh
1,Pig Meat,Supplied for Consumption,10.51,24.14,10.88,21.69,22.29,27.64,16.84,43.58,...,0.18,0.28,13,5.13,6.52,1.22,0.05,2.98,4.01,0
2,Pig Meat,Actually Consumed,37.2,85.44,38.51,76.77,78.9,97.83,59.61,154.25,...,0.64,0.99,46.01,18.16,23.08,4.32,0.18,10.55,14.19,0
3,Poultry Meat,Supplied for Consumption,38.66,46.12,13.23,26.87,34.98,50.01,27.45,21.37,...,2.66,1.88,13.69,1.41,1.84,4.97,4.96,7.5,8.91,1.4
4,Poultry Meat,Actually Consumed,41.53,49.54,14.21,28.86,37.58,53.72,29.49,22.96,...,2.86,2.02,14.71,1.51,1.98,5.34,5.33,8.06,9.57,1.5


In [18]:
#grab the first row for the header
new_header = countries_df.iloc[0]
#take only the data below the first row
countries_df = countries_df[1:]
#set the header row as the df header
countries_df.columns = new_header
countries_df.head()


,Food product,Country,Argentina,Australia,Albania,Iceland,New Zealand,USA,Uruguay,Luxembourg,...,Gambia,India,Thailand,Mozambique,Cambodia,Sierra Leone,Sri Lanka,Indonesia,Liberia,Bangladesh
1,Pig Meat,Supplied for Consumption,10.51,24.14,10.88,21.69,22.29,27.64,16.84,43.58,...,0.18,0.28,13,5.13,6.52,1.22,0.05,2.98,4.01,0
2,Pig Meat,Actually Consumed,37.2,85.44,38.51,76.77,78.9,97.83,59.61,154.25,...,0.64,0.99,46.01,18.16,23.08,4.32,0.18,10.55,14.19,0
3,Poultry Meat,Supplied for Consumption,38.66,46.12,13.23,26.87,34.98,50.01,27.45,21.37,...,2.66,1.88,13.69,1.41,1.84,4.97,4.96,7.5,8.91,1.4
4,Poultry Meat,Actually Consumed,41.53,49.54,14.21,28.86,37.58,53.72,29.49,22.96,...,2.86,2.02,14.71,1.51,1.98,5.34,5.33,8.06,9.57,1.5
5,Beef (beef herd),Supplied for Consumption,55.48,33.86,22.5,13.36,22.49,36.24,29.1,29.88,...,2.16,0.81,2.56,1.04,4.85,1.99,1.38,2.59,0.78,1.28


In [19]:
countries_df = countries_df.rename(columns={"Country":"Supplied or Consumed (kg CO2/person/year)"})
countries_df.head()


,Food product,Supplied or Consumed (kg CO2/person/year),Argentina,Australia,Albania,Iceland,New Zealand,USA,Uruguay,Luxembourg,...,Gambia,India,Thailand,Mozambique,Cambodia,Sierra Leone,Sri Lanka,Indonesia,Liberia,Bangladesh
1,Pig Meat,Supplied for Consumption,10.51,24.14,10.88,21.69,22.29,27.64,16.84,43.58,...,0.18,0.28,13,5.13,6.52,1.22,0.05,2.98,4.01,0
2,Pig Meat,Actually Consumed,37.2,85.44,38.51,76.77,78.9,97.83,59.61,154.25,...,0.64,0.99,46.01,18.16,23.08,4.32,0.18,10.55,14.19,0
3,Poultry Meat,Supplied for Consumption,38.66,46.12,13.23,26.87,34.98,50.01,27.45,21.37,...,2.66,1.88,13.69,1.41,1.84,4.97,4.96,7.5,8.91,1.4
4,Poultry Meat,Actually Consumed,41.53,49.54,14.21,28.86,37.58,53.72,29.49,22.96,...,2.86,2.02,14.71,1.51,1.98,5.34,5.33,8.06,9.57,1.5
5,Beef (beef herd),Supplied for Consumption,55.48,33.86,22.5,13.36,22.49,36.24,29.1,29.88,...,2.16,0.81,2.56,1.04,4.85,1.99,1.38,2.59,0.78,1.28


In [20]:
countries_df = countries_df.loc[countries_df["Supplied or Consumed (kg CO2/person/year)"] != "Supplied for Consumption"]
countries_df


,Food product,Supplied or Consumed (kg CO2/person/year),Argentina,Australia,Albania,Iceland,New Zealand,USA,Uruguay,Luxembourg,...,Gambia,India,Thailand,Mozambique,Cambodia,Sierra Leone,Sri Lanka,Indonesia,Liberia,Bangladesh
2,Pig Meat,Actually Consumed,37.2,85.44,38.51,76.77,78.9,97.83,59.61,154.25,...,0.64,0.99,46.01,18.16,23.08,4.32,0.18,10.55,14.19,0
4,Poultry Meat,Actually Consumed,41.53,49.54,14.21,28.86,37.58,53.72,29.49,22.96,...,2.86,2.02,14.71,1.51,1.98,5.34,5.33,8.06,9.57,1.5
6,Beef (beef herd),Actually Consumed,1712,1044.85,694.3,412.26,693.99,1118.29,897.96,922.03,...,66.65,24.99,79,32.09,149.66,61.41,42.58,79.92,24.07,39.5
8,Lamb & Mutton,Actually Consumed,54.63,345.65,536.5,739.62,662.23,15.06,288.21,58.48,...,24.86,20.31,1.75,31.52,0,22.76,3.15,16.11,16.81,46.58
10,Fish (farmed),Actually Consumed,6.96,28.25,6.15,118.81,32.51,19.72,10.43,36.87,...,36.93,7.52,31.06,12.36,63.21,51.14,39.79,40.95,6.59,28.85
12,Eggs,Actually Consumed,10.46,7.82,11.44,7.57,9.1,13.39,12.07,13.45,...,0.91,2.37,11.34,1.56,1.26,1.76,4.2,4.48,1.88,1.91
14,Dairy,Actually Consumed,277.87,334.01,432.62,321.66,195.5,362.78,299.89,363.65,...,65.11,120.36,41.81,6.82,4.94,9.97,49.83,21.11,4.33,31.21
15,,Total Kg CO2/person/year for Animal Products,2140.65,1895.55,1733.73,1705.55,1709.8,1680.79,1597.66,1571.69,...,197.96,178.57,225.67,104.03,244.13,156.7,145.05,181.18,77.45,149.55
17,Wheat and Wheat Products,Actually Consumed,19.66,13.44,26.44,13.91,14.67,15.34,20.85,19.68,...,7.23,11.55,2.08,2.79,0.52,1.66,7.06,4.85,2.09,3.33
19,Rice,Actually Consumed,11.22,14.12,9.96,4.98,11.72,8.8,14.72,5.37,...,79.14,88.93,146.62,29.96,203.6,132.19,140.41,172.27,121.25,219.76


In [21]:
del countries_df["Supplied or Consumed (kg CO2/person/year)"]
countries_df


,Food product,Argentina,Australia,Albania,Iceland,New Zealand,USA,Uruguay,Luxembourg,Brazil,...,Gambia,India,Thailand,Mozambique,Cambodia,Sierra Leone,Sri Lanka,Indonesia,Liberia,Bangladesh
2,Pig Meat,37.2,85.44,38.51,76.77,78.9,97.83,59.61,154.25,44.6,...,0.64,0.99,46.01,18.16,23.08,4.32,0.18,10.55,14.19,0
4,Poultry Meat,41.53,49.54,14.21,28.86,37.58,53.72,29.49,22.96,48.34,...,2.86,2.02,14.71,1.51,1.98,5.34,5.33,8.06,9.57,1.5
6,Beef (beef herd),1712,1044.85,694.3,412.26,693.99,1118.29,897.96,922.03,1211.17,...,66.65,24.99,79,32.09,149.66,61.41,42.58,79.92,24.07,39.5
8,Lamb & Mutton,54.63,345.65,536.5,739.62,662.23,15.06,288.21,58.48,21.71,...,24.86,20.31,1.75,31.52,0,22.76,3.15,16.11,16.81,46.58
10,Fish (farmed),6.96,28.25,6.15,118.81,32.51,19.72,10.43,36.87,15.98,...,36.93,7.52,31.06,12.36,63.21,51.14,39.79,40.95,6.59,28.85
12,Eggs,10.46,7.82,11.44,7.57,9.1,13.39,12.07,13.45,8.25,...,0.91,2.37,11.34,1.56,1.26,1.76,4.2,4.48,1.88,1.91
14,Dairy,277.87,334.01,432.62,321.66,195.5,362.78,299.89,363.65,212.63,...,65.11,120.36,41.81,6.82,4.94,9.97,49.83,21.11,4.33,31.21
15,,2140.65,1895.55,1733.73,1705.55,1709.8,1680.79,1597.66,1571.69,1562.69,...,197.96,178.57,225.67,104.03,244.13,156.7,145.05,181.18,77.45,149.55
17,Wheat and Wheat Products,19.66,13.44,26.44,13.91,14.67,15.34,20.85,19.68,10.11,...,7.23,11.55,2.08,2.79,0.52,1.66,7.06,4.85,2.09,3.33
19,Rice,11.22,14.12,9.96,4.98,11.72,8.8,14.72,5.37,41.12,...,79.14,88.93,146.62,29.96,203.6,132.19,140.41,172.27,121.25,219.76


In [126]:
merged_df = pd.merge(countries_df, food_df)
merged_df


,Food product,Argentina,Australia,Albania,Iceland,New Zealand,USA,Uruguay,Luxembourg,Brazil,...,Indonesia,Liberia,Bangladesh,Land use change,Animal Feed,Farm,Processing,Transport,Packging,Retail
0,Pig Meat,37.2,85.44,38.51,76.77,78.9,97.83,59.61,154.25,44.6,...,10.55,14.19,0,1.5,2.9,1.7,0.3,0.3,0.3,0.2
1,Poultry Meat,41.53,49.54,14.21,28.86,37.58,53.72,29.49,22.96,48.34,...,8.06,9.57,1.5,2.5,1.8,0.7,0.4,0.3,0.2,0.2
2,Beef (beef herd),1712,1044.85,694.3,412.26,693.99,1118.29,897.96,922.03,1211.17,...,79.92,24.07,39.5,16.3,1.9,39.4,1.3,0.3,0.2,0.2
3,Lamb & Mutton,54.63,345.65,536.5,739.62,662.23,15.06,288.21,58.48,21.71,...,16.11,16.81,46.58,0.5,2.4,19.5,1.1,0.5,0.3,0.2
4,Fish (farmed),6.96,28.25,6.15,118.81,32.51,19.72,10.43,36.87,15.98,...,40.95,6.59,28.85,0.5,0.8,3.6,0.0,0.1,0.1,0.0
5,Eggs,10.46,7.82,11.44,7.57,9.1,13.39,12.07,13.45,8.25,...,4.48,1.88,1.91,0.7,2.2,1.3,0.0,0.1,0.2,0.0
6,Dairy,277.87,334.01,432.62,321.66,195.5,362.78,299.89,363.65,212.63,...,21.11,4.33,31.21,5.0,2.5,14.6,0.8,0.2,0.3,0.6
7,Wheat and Wheat Products,19.66,13.44,26.44,13.91,14.67,15.34,20.85,19.68,10.11,...,4.85,2.09,3.33,0.1,0.0,0.8,0.2,0.1,0.1,0.1
8,Rice,11.22,14.12,9.96,4.98,11.72,8.8,14.72,5.37,41.12,...,172.27,121.25,219.76,0.0,0.0,3.6,0.1,0.1,0.1,0.1
9,Nuts,0.87,15.45,7.72,6.87,14.55,13.91,1.68,1.65,1.19,...,8.71,2.32,1.27,-1.7,0.0,3.5,0.4,0.2,0.2,0.0


In [127]:
merged_df = merged_df.transpose()
merged_df


,0,1,2,3,4,5,6,7,8,9
Food product,Pig Meat,Poultry Meat,Beef (beef herd),Lamb & Mutton,Fish (farmed),Eggs,Dairy,Wheat and Wheat Products,Rice,Nuts
Argentina,37.2,41.53,1712,54.63,6.96,10.46,277.87,19.66,11.22,0.87
Australia,85.44,49.54,1044.85,345.65,28.25,7.82,334.01,13.44,14.12,15.45
Albania,38.51,14.21,694.3,536.5,6.15,11.44,432.62,26.44,9.96,7.72
Iceland,76.77,28.86,412.26,739.62,118.81,7.57,321.66,13.91,4.98,6.87
...,...,...,...,...,...,...,...,...,...,...
Farm,1.7,0.7,39.4,19.5,3.6,1.3,14.6,0.8,3.6,3.5
Processing,0.3,0.4,1.3,1.1,0,0,0.8,0.2,0.1,0.4
Transport,0.3,0.3,0.3,0.5,0.1,0.1,0.2,0.1,0.1,0.2
Packging,0.3,0.2,0.2,0.3,0.1,0.2,0.3,0.1,0.1,0.2


In [128]:
merged_df = merged_df.reset_index()
merged_df


,index,0,1,2,3,4,5,6,7,8,9
0,Food product,Pig Meat,Poultry Meat,Beef (beef herd),Lamb & Mutton,Fish (farmed),Eggs,Dairy,Wheat and Wheat Products,Rice,Nuts
1,Argentina,37.2,41.53,1712,54.63,6.96,10.46,277.87,19.66,11.22,0.87
2,Australia,85.44,49.54,1044.85,345.65,28.25,7.82,334.01,13.44,14.12,15.45
3,Albania,38.51,14.21,694.3,536.5,6.15,11.44,432.62,26.44,9.96,7.72
4,Iceland,76.77,28.86,412.26,739.62,118.81,7.57,321.66,13.91,4.98,6.87
...,...,...,...,...,...,...,...,...,...,...,...
133,Farm,1.7,0.7,39.4,19.5,3.6,1.3,14.6,0.8,3.6,3.5
134,Processing,0.3,0.4,1.3,1.1,0,0,0.8,0.2,0.1,0.4
135,Transport,0.3,0.3,0.3,0.5,0.1,0.1,0.2,0.1,0.1,0.2
136,Packging,0.3,0.2,0.2,0.3,0.1,0.2,0.3,0.1,0.1,0.2


In [129]:
#grab the first row for the header
new_header = merged_df.iloc[0]
#take only the data below the first row
merged_df = merged_df[1:]
#set the header row as the df header
merged_df.columns = new_header
merged_df


,Food product,Pig Meat,Poultry Meat,Beef (beef herd),Lamb & Mutton,Fish (farmed),Eggs,Dairy,Wheat and Wheat Products,Rice,Nuts
1,Argentina,37.2,41.53,1712,54.63,6.96,10.46,277.87,19.66,11.22,0.87
2,Australia,85.44,49.54,1044.85,345.65,28.25,7.82,334.01,13.44,14.12,15.45
3,Albania,38.51,14.21,694.3,536.5,6.15,11.44,432.62,26.44,9.96,7.72
4,Iceland,76.77,28.86,412.26,739.62,118.81,7.57,321.66,13.91,4.98,6.87
5,New Zealand,78.9,37.58,693.99,662.23,32.51,9.1,195.5,14.67,11.72,14.55
...,...,...,...,...,...,...,...,...,...,...,...
133,Farm,1.7,0.7,39.4,19.5,3.6,1.3,14.6,0.8,3.6,3.5
134,Processing,0.3,0.4,1.3,1.1,0,0,0.8,0.2,0.1,0.4
135,Transport,0.3,0.3,0.3,0.5,0.1,0.1,0.2,0.1,0.1,0.2
136,Packging,0.3,0.2,0.2,0.3,0.1,0.2,0.3,0.1,0.1,0.2


In [130]:
merged_df = merged_df.rename(columns={"Food product":"country_or_process",
                                     "Pig Meat":"pig", "Poultry Meat":"poultry", "Beef (beef herd)":"beef",
                                     "Lamb & Mutton":"lamb_and_mutton", "Fish (farmed)":"farmed_fish", "Eggs":"eggs",
                                     "Dairy":"dairy", "Wheat and Wheat Products":"wheat_products", "Rice":"rice", "Nuts":"nuts"})
merged_df


,country_or_process,pig,poultry,beef,lamb_and_mutton,farmed_fish,eggs,dairy,wheat_products,rice,nuts
1,Argentina,37.2,41.53,1712,54.63,6.96,10.46,277.87,19.66,11.22,0.87
2,Australia,85.44,49.54,1044.85,345.65,28.25,7.82,334.01,13.44,14.12,15.45
3,Albania,38.51,14.21,694.3,536.5,6.15,11.44,432.62,26.44,9.96,7.72
4,Iceland,76.77,28.86,412.26,739.62,118.81,7.57,321.66,13.91,4.98,6.87
5,New Zealand,78.9,37.58,693.99,662.23,32.51,9.1,195.5,14.67,11.72,14.55
...,...,...,...,...,...,...,...,...,...,...,...
133,Farm,1.7,0.7,39.4,19.5,3.6,1.3,14.6,0.8,3.6,3.5
134,Processing,0.3,0.4,1.3,1.1,0,0,0.8,0.2,0.1,0.4
135,Transport,0.3,0.3,0.3,0.5,0.1,0.1,0.2,0.1,0.1,0.2
136,Packging,0.3,0.2,0.2,0.3,0.1,0.2,0.3,0.1,0.1,0.2


In [138]:
merged_df = merged_df.reset_index()
merged_df


,index,country_or_process,pig,poultry,beef,lamb_and_mutton,farmed_fish,eggs,dairy,wheat_products,rice,nuts
0,1,Argentina,37.2,41.53,1712,54.63,6.96,10.46,277.87,19.66,11.22,0.87
1,2,Australia,85.44,49.54,1044.85,345.65,28.25,7.82,334.01,13.44,14.12,15.45
2,3,Albania,38.51,14.21,694.3,536.5,6.15,11.44,432.62,26.44,9.96,7.72
3,4,Iceland,76.77,28.86,412.26,739.62,118.81,7.57,321.66,13.91,4.98,6.87
4,5,New Zealand,78.9,37.58,693.99,662.23,32.51,9.1,195.5,14.67,11.72,14.55
...,...,...,...,...,...,...,...,...,...,...,...,...
132,133,Farm,1.7,0.7,39.4,19.5,3.6,1.3,14.6,0.8,3.6,3.5
133,134,Processing,0.3,0.4,1.3,1.1,0,0,0.8,0.2,0.1,0.4
134,135,Transport,0.3,0.3,0.3,0.5,0.1,0.1,0.2,0.1,0.1,0.2
135,136,Packging,0.3,0.2,0.2,0.3,0.1,0.2,0.3,0.1,0.1,0.2


In [143]:
merged_df = merged_df.rename(columns={"index":"id"})
merged_df


,id,country_or_process,pig,poultry,beef,lamb_and_mutton,farmed_fish,eggs,dairy,wheat_products,rice,nuts
0,1,Argentina,37.2,41.53,1712,54.63,6.96,10.46,277.87,19.66,11.22,0.87
1,2,Australia,85.44,49.54,1044.85,345.65,28.25,7.82,334.01,13.44,14.12,15.45
2,3,Albania,38.51,14.21,694.3,536.5,6.15,11.44,432.62,26.44,9.96,7.72
3,4,Iceland,76.77,28.86,412.26,739.62,118.81,7.57,321.66,13.91,4.98,6.87
4,5,New Zealand,78.9,37.58,693.99,662.23,32.51,9.1,195.5,14.67,11.72,14.55
...,...,...,...,...,...,...,...,...,...,...,...,...
132,133,Farm,1.7,0.7,39.4,19.5,3.6,1.3,14.6,0.8,3.6,3.5
133,134,Processing,0.3,0.4,1.3,1.1,0,0,0.8,0.2,0.1,0.4
134,135,Transport,0.3,0.3,0.3,0.5,0.1,0.1,0.2,0.1,0.1,0.2
135,136,Packging,0.3,0.2,0.2,0.3,0.1,0.2,0.3,0.1,0.1,0.2


In [134]:
# Create database connection
rds_connection_string = f'{username}:{password}@localhost:5432/emissions_db'
engine = create_engine(f'postgresql://{rds_connection_string}')


In [135]:
# Confirm tables
engine.table_names()

['country_food_emissions']

In [145]:
merged_df.to_sql(name='country_food_emissions', con=engine, if_exists='append', index=False)


In [147]:
pd.read_sql_query('select * from country_food_emissions', con=engine)


,id,country_or_process,pig,poultry,beef,lamb_and_mutton,farmed_fish,eggs,dairy,wheat_products,rice,nuts
0,1,Argentina,37.20,41.53,1712.00,54.63,6.96,10.46,277.87,19.66,11.22,0.87
1,2,Australia,85.44,49.54,1044.85,345.65,28.25,7.82,334.01,13.44,14.12,15.45
2,3,Albania,38.51,14.21,694.30,536.50,6.15,11.44,432.62,26.44,9.96,7.72
3,4,Iceland,76.77,28.86,412.26,739.62,118.81,7.57,321.66,13.91,4.98,6.87
4,5,New Zealand,78.90,37.58,693.99,662.23,32.51,9.10,195.50,14.67,11.72,14.55
...,...,...,...,...,...,...,...,...,...,...,...,...
132,133,Farm,1.70,0.70,39.40,19.50,3.60,1.30,14.60,0.80,3.60,3.50
133,134,Processing,0.30,0.40,1.30,1.10,0.00,0.00,0.80,0.20,0.10,0.40
134,135,Transport,0.30,0.30,0.30,0.50,0.10,0.10,0.20,0.10,0.10,0.20
135,136,Packging,0.30,0.20,0.20,0.30,0.10,0.20,0.30,0.10,0.10,0.20
